In [57]:
from sklearn.datasets import load_boston
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from sklearn import datasets, linear_model
from sklearn import metrics, cross_validation

In [58]:
boston = load_boston()
print(boston.data.shape)

(506, 13)


In [59]:
type(boston)

sklearn.datasets.base.Bunch

In [60]:
print(boston.DESCR)
print(boston.keys())
print(boston.feature_names)
print(boston.data.shape)

Boston House Prices dataset

Notes
------
Data Set Characteristics:  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive
    
    :Median Value (attribute 14) is usually the target

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pupil-teacher ratio by town
      

In [61]:
X = boston.data

In [62]:
X[0:2, :]

array([[  6.32000000e-03,   1.80000000e+01,   2.31000000e+00,
          0.00000000e+00,   5.38000000e-01,   6.57500000e+00,
          6.52000000e+01,   4.09000000e+00,   1.00000000e+00,
          2.96000000e+02,   1.53000000e+01,   3.96900000e+02,
          4.98000000e+00],
       [  2.73100000e-02,   0.00000000e+00,   7.07000000e+00,
          0.00000000e+00,   4.69000000e-01,   6.42100000e+00,
          7.89000000e+01,   4.96710000e+00,   2.00000000e+00,
          2.42000000e+02,   1.78000000e+01,   3.96900000e+02,
          9.14000000e+00]])

In [63]:
y = boston.target

In [64]:
y[0:2]

array([ 24. ,  21.6])

## Prepare train/test sets

In [65]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size = 0.33, random_state = 5)

print("X_train shape: " + str(X_train.shape))
print("y_train shape: " + str(y_train.shape))
print("X_test shape: " + str(X_test.shape))
print("y_test shape: " + str(y_test.shape))

X_train shape: (339, 13)
y_train shape: (339,)
X_test shape: (167, 13)
y_test shape: (167,)


## Define functions

In [10]:
def sum_squared_error(X, y, theta):
    hypothesis = X.dot(theta.T)
    square_of_errors = np.square(hypothesis - y)
    return np.sum(square_of_errors)

def mean_squared_error(X, y, theta):
    m = X.shape[0]
    return (1 / (2 * m)) * sum_squared_error(X, y, theta)
    
def gradient(X, y, theta):
    m = X.shape[0]
    hypothesis = X.dot(theta.T)
    error = hypothesis - y
    gradient = (1/m) * X.T.dot(error)
    return (gradient, error)
    
def gradient_descent_batch(X, y, theta, alpha, iterations):
    m = X.shape[0]
    for i in range(iterations):
        # Move in opposite direction to gradient, hence minus gradient
        # Alpha seems to temper the gradient change, preventing huge swings. It should be low.
        grad, error = gradient(X, y, theta)
        theta = theta - (alpha * grad)
    
        if (debug):
            print("Iteration: " + str(i))    
            print("  Theta: " + str(theta))
            print("  Error: " + str(error))
            
    return theta

def gradient_descent_stochastic(X, y, theta, alpha, iterations):
    print("To Do")

### Testing functions ###
debug = True    

t_X = np.array([[1, 2, 3],[4, 5, 6]])
t_y = [6, 15]
t_theta = np.array([2, 1, 1])
print("Theta: " + str(t_theta))

# Test cost function
mse = mean_squared_error(t_X, t_y, t_theta)
print("Cost: " + str(mse))
    
# Test gradient function
g, e = gradient(t_X, t_y, t_theta)
print("Gradient: " + str(g))

# Test gradient descent batch
t_initial_theta = np.array([4,3,2])
optimised_theta = gradient_descent_batch(t_X, t_y, t_initial_theta, 0.01, 500)
optmised_theta_mse = mean_squared_error(t_X, t_y, optimised_theta)
print("Optimised theta: " + str(optimised_theta))
print("Optimised theta cost: " + str(optmised_theta_mse))

Theta: [2 1 1]
Cost: 4.25
Gradient: [  8.5  11.   13.5]
Iteration: 0
  Theta: [ 3.39  2.2   1.01]
  Error: [10 28]
Iteration: 1
  Theta: [ 3.0535  1.7613  0.4691]
  Error: [  4.82  15.62]
Iteration: 2
  Theta: [ 2.866881   1.5205885  0.174296 ]
  Error: [ 1.9834  8.8351]
Iteration: 3
  Theta: [ 2.76240142  1.38837298  0.01434453]
  Error: [ 0.430946   5.1162425]
Iteration: 4
  Theta: [ 2.70293976  1.31561272 -0.07171431]
  Error: [-0.41781902  3.07753778]
Iteration: 5
  Theta: [ 2.66815391  1.27543408 -0.11728575]
  Error: [-0.88097773  1.95953678]
Iteration: 6
  Theta: [ 2.64689666  1.25311064 -0.14067537]
  Error: [-1.13283518  1.34607155]
Iteration: 7
  Theta: [ 2.63305945  1.24057253 -0.15191438]
  Error: [-1.26890816  1.00908763]
Iteration: 8
  Theta: [ 2.62329485  1.23339757 -0.15649972]
  Error: [-1.34153861  0.8236142 ]
Iteration: 9
  Theta: [ 2.61576852  1.22916243 -0.15744365]
  Error: [-1.37940918  0.72116891]
Iteration: 10
  Theta: [ 2.60947522  1.2265392  -0.15639682]
  Er

## Run Linear Regression From Scikit-learn

<b>Provides some results to compare against</b>

In [53]:
regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)

print('Coefficients: \n', regr.coef_)
print("Mean squared error (train): %.2f" % metrics.mean_squared_error(y_train, regr.predict(X_train)))
print("Mean squared error (test): %.2f" % metrics.mean_squared_error(y_test, y_pred))

Coefficients: 
 [ -1.56381297e-01   3.85490972e-02  -2.50629921e-02   7.86439684e-01
  -1.29469121e+01   4.00268857e+00  -1.16023395e-02  -1.36828811e+00
   3.41756915e-01  -1.35148823e-02  -9.88866034e-01   1.20588215e-02
  -4.72644280e-01]
Mean squared error (train): 19.55
Mean squared error (test): 28.54


## Run Linear Regression With Full Batch Gradient Descent

In [56]:
debug = False
num_of_iterations = 100000
alpha = 0.000001 # learning rate
initial_theta = np.ones((X.shape[1]))

optimised_theta = gradient_descent_batch(X_train, y_train, initial_theta, alpha, num_of_iterations)
mse_on_train = mean_squared_error(X_train, y_train, optimised_theta)
mse_on_test = mean_squared_error(X_test, y_test, optimised_theta)
print("Mean squared error (train): " + str(mse_on_train))
print("Mean squared error (test): " + str(mse_on_test))
print(optimised_theta)

Mean squared error (train): 20.4148990639
Mean squared error (test): 24.2765628858
[-0.16782881  0.0886393   0.26042914  1.0036959   1.00358331  1.311048
  0.08550841  0.7656243   0.56757561 -0.03132915  0.48488266  0.02434233
 -0.71187595]


## Run Linear Regression With Stochastic Gradient Descent